In [19]:
%cd '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect'

/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect


In [20]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import cv2
import os
import sys
import numpy as np
import random

sys.path.append(os.getcwd())

from cnn import make_model

In [4]:
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'models/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

def define_stacked_model(neural_nets):
	# update all layers in all models to not be trainable
	for i in range(len(neural_nets)):
		model = neural_nets[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer.name = 'ensemble_' + str(i+1) + '_' + layer.name
	# define multi-headed input
	ensemble_visible = [model.input for model in neural_nets]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in neural_nets]
    
	merge = concatenate(ensemble_outputs)
	hidden = layer.Dense(10, activation='relu')(merge)
	output = layer.Dense(3, activation='softmax')(hidden)
	model = Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='data/plots/model_graph.png')
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	model.fit(X, inputy_enc, epochs=300, verbose=0)
 
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [28]:
checkpoint_path = 'categorization/model_saves/checkpoint'
# checkpoint_dir = os.path.dirname(checkpoint_path)
model = make_model(217)
# latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(checkpoint_path)


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for categorization/model_saves/checkpoint